This portion of the code imports the sim files from the directory containing the configuration file, and the BurstCube module is used to extract elements from the files: energy, azimuth, and zenith. 
Astropy module is also imported to allow for creation of tables/updating tables.


In [1]:
# import required modules for making a table and loading in BurstCube
# software
from astropy.table import Table
from BurstCube.simGenerator import configurator
from BurstCube.bcSim import simFiles

# configuration file taken from chosen directory to load in sim files 
# (start from the current directory)
localFile = input('Enter your file name:')
conf = configurator(localFile)

# set all simfiles as sfs
sfs = simFiles(localFile)

# use area variable to extract needed elements from simulation files
# : azimuth, zenith, and energy
area = sfs.calculateAeff()

Energy = list(area['keV'])
Azimuth = list(area['az'])
Zenith = list(area['ze'])

Enter your file name:randomBursts/315_100_60/config.yaml
Loading /home/laura/Research/local_test/SmallSteps/finalTest/randomBursts/315_100_60//31510060_100.000keV_60.00ze_315.00az.inc1.id1.sim
[60.0]


The detector counting variables are initialized per each sim file. The lists hitsData is also initialized to store all data (either from one file to find the unknown location of a random burst or from multiple files to store simulated data in a table).

Detector counts are found by extracting lines out of simulation files containing hit data (each hit should be located on the center of a detector because the source files set the DiscretizeHits variable to TRUE):
* each detector is located in each quadrant of the cartesian plane
* the azimuth angle is measured counterclockwise from the x-axis
* the absolute value of each center x-y pair is: (5.52500, 5.52500)

In [2]:
count = 0; prevLine = 'random'
hitsData = []; trigger = []

# import required modules
import re
import sys
import numpy as np

# loop through all sim files
for file in sfs.sims:
    
    # initialize variables used to sum hits per detector 
    counter1 = 0; counter2 = 0; counter3 = 0; counter4 = 0
    
    # open each simulation file and loop over all lines
    with open(file.simFile) as sim:
#         print(file.simFile)
        for i, line in enumerate(sim):
            
            # this code is not yet finished, but if the ID would
            # be needed, this portion would go to use
            if line.startswith('ID'):
                ID = line.split()                  
                    
            # split line by semi-colons/spaces to retrieve hit positions (detector centers)
            # to determine which detector is being hit
            if line.startswith('HTsim'):
                
                data = line.split(";")
                data = [w.strip(' ') for w in data]
                # get the x-coordinate of the detector center where the hit occurs
                data[1] = float(data[1])
                # get the y-coordinate of the detector center where the hit occurs
                data[2] = float(data[2])

                #detector 1
                if (abs(5.52500 - data[1]) <= 0.05) and  (abs(5.52500 - data[2]) <= .05):
                    counter1 += 1
#                     print('1: ', counter1)
                #detector 2    
                elif (abs(-5.52500 - data[1]) <= 0.05) and  (abs(5.52500 - data[2]) <= .05):
                    counter2 += 1
#                     print('2: ', counter2)
                #detector 3 
                elif (abs(-5.52500 - data[1]) <= 0.05) and  (abs(-5.52500 - data[2]) <= .05):
                    counter3 += 1
#                     print('3: ', counter3)
                #detector 4
                elif (abs(5.52500 - data[1]) <= 0.05) and  (abs(-5.52500 - data[2]) <= .05):
                    counter4 += 1
#                     print('4: ', counter4) 
            
            # (not yet completed): if the previous line starts with HTsim, record that double occurence
            if prevLine.startswith('HTsim') and line.startswith('HTsim'):
                count += 1
                
#                 if fileCount == '0':
#                     tableNames = ('Energy', 'Azimuth', 'Zenith', 'triggerID1', 'triggerID2', 'tallyOfOccurence')
#                     dt = Table(Energy[fileCount], Azimuth[fileCount], Zenith[fileCount], prevTrigger, 
#                               trigger[counter], count, names=tableNames)
#                     dt.write('doubleTriggerTable.txt', format='ascii.ecsv')
                    
#                 else:
#                     dt.add_row(Energy[fileCount], Azimuth[fileCount], Zenith[fileCount], prevTrigger, 
#                               trigger[counter], count, names=tableNames)
#                     dt.write('doubleTriggerTable.txt', format='ascii.ecsv')
                
                sfo = re.split('[_ . //]', file.simFile)
    
            prevLine = line           
                
    # append a list to hitsData for the total number of hits per each detector
    hitsData.append([counter1, counter2, counter3, counter4])


Enter 1 if this simulation is for the new localization code or enter 0 for old code:1


The small cell located below helps to analyze the hitsData array for future use.

In [3]:
print(hitsData)

[[254, 156, 227, 363]]


This cell has multiple functions:
* First, the user is asked whether an uncertainty factor will be applied to the data (if so, the entire cell is bypassed)
* If not, a series of statements allows the user to choose if they want to either determine the random burst location (inputDet will have the same value as hitsData), store simulated data loaded in from hitsData into a dictionary, or bypass all options.
* In the case that a dictionary is made, all elements partaining to the burst data are extracted: the energy, azimuth, zenith, and the numbers of hits per each detector (all per each simulation file)

In [4]:
# if using the uncertainty calculation to gain accuracy in stored simulated GRB data
uncertainty = input('If you are storing uncertainty data, enter 1: ')
if uncertainty == '1':
    pass

else:
    # if using new code, examine the options below
    # if GRBfinder == '1':
        
        randomB = input('Enter 1 if you want to determine the burst location, 0 to store data new data in a dict, and anything else to pass: ')
        
        # to determine what azimuth corresponds to the random generated GRB
        if randomB == '1':
            inputDet = hitsData
        
        # to take new simulated GRB data to be stored in table, first store that data in a dictionary
        elif randomB == '0':
            hits = np.array(hitsData)
            
            # intialize dictionary, storing energy, azimuth, zenith, and hits per detector
            LocalizationDict2 = {}
            LocalizationDict2.update({'Energy': Energy})
            LocalizationDict2.update({'Azimuth': Azimuth})
            LocalizationDict2.update({'Zenith': Zenith})

            LocalizationDict2['hits1'] =[]
            LocalizationDict2['hits2'] = []
            LocalizationDict2['hits3']= []
            LocalizationDict2['hits4']= []
            
            # each list takes only the detector hits that correspond
            # to its dictionary key title (ex: 'hits1' means all data for detector one)
            for x in list(hits[:,0]):
                LocalizationDict2['hits1'].append(x)
            for y in list(hits[:,1]):
                LocalizationDict2['hits2'].append(y)
            for z in list(hits[:,2]):
                LocalizationDict2['hits3'].append(z)
            for q in list(hits[:,3]):
                LocalizationDict2['hits4'].append(q)
        else:
            pass

If you are storing uncertainty data, enter 1: 0
Enter 1 if you want to determine the burst location, 0 to store data new data in a dict, and anything else to pass: 1


The user is then asked another set of options:
* If they want to use a table to store data, they can either choose to make a table for the first time or load in new data from dictionary, adding a new row in the table per each set of data in dictionary (the table is preprogrammed to have the same titles as the dictionary to make merging easy)
* They could also just read in the data already stored in the table if they would like to apply an uncertainty factor on select data

In [5]:
start = input('If making the data table for the first time enter 0, enter 1 for updating new code, enter 2 to read in normal data, or enter 3 to read in temp data: ')

# if making table for the first time, use dictionary to 
# import values and col names
if start == '0':
    colnames = ('Energy', 'Azimuth', 'Zenith', 'hits1', 'hits2', 'hits3', 'hits4')
    t = Table(LocalizationDict2, names=colnames)
    t.write('storedBurstData.txt', format='ascii.ecsv')

# addding data to table, use dictionary and add new row per simulation   
elif start == '1':
    t = Table.read('storedBurstData.txt', format='ascii.ecsv')
    for i in range(0, len(hitsData)):
        t.add_row([LocalizationDict2['Energy'][i], LocalizationDict2['Azimuth'][i],
        LocalizationDict2['Zenith'][i], LocalizationDict2['hits1'][i],
        LocalizationDict2['hits2'][i], LocalizationDict2['hits3'][i],
        LocalizationDict2['hits4'][i]])
        t.write('storedBurstData.txt', format='ascii.ecsv')

# loading stored data to conduct uncertainty calculation
elif start == '2':
    t = Table.read('storedBurstData.txt', format='ascii.ecsv')

# loading data stored in temp folder to see how uncertainty data compares to the random
# burst on its own
else # start == '3':
    t = Table.read('tempBurstData.txt', format='ascii.ecsv')


If making the data table for the first time enter 0, enter 1 for updating new code, enter 2 to read in normal data, enter 3 to work with temp data, enter 4 to read in test data, and enter 5 to store test data: 2


If the uncertainty factor is applied, an array variable trialArrayOH will contain all rows in the loaded table that have an energy of 100 keV. The table range to be extracted will be requested (using "first" and "second"), and then the number of hits per detector will be loaded from that data into the hitsArray variable. A list will then be generated corresponding to each set of detector data, and each element in each list will have the uncertainty factor applied, which is the square root of the number of triggers over the number of triggers (0.0316227766 or about 3 percent). The step size pertaining to the range of uncertainty, from a low of 3 percent lower than the current value to 3 percent higher than the current value, will also be inputted from the user. Uncertainty data will then be stored in a table.

In [6]:
if uncertainty == '1':
    # create list variables corresponding to each detector
    l1 = []
    l2 = []
    l3 = []
    l4 = []
    uncertAngle = []
    
    # generate array of data corresponding to all values with 
    # an energy of 100 keV
    trialArrayOH = t[[t['Energy']==100]]
    first = int(input('Enter the index in the table for the first number in the trialArrayOH range: '))
    second = int(input('Enter the index in the table for the second number in the trialArrayOH range: '))
    
    # add a correction factor to make sure the table values line up
    first = first - 13
    second = second - 12

    trialArrayOH = trialArrayOH[first:second]
    print(trialArrayOH)
    
    # extract hits per each detector
    hitsArray = np.array([trialArrayOH['hits1'], trialArrayOH['hits2'], 
                                      trialArrayOH['hits3'], trialArrayOH['hits4']])

    def output_list(num, numDivisions):
        """Generates hits per detector data with uncertainty applied

        Parameters
        ----------
        num : the number of hits for a certain detector occurence
        
        numDivisions : the number of partitions between the lower uncertainty and 
                       higher uncertainty

        Returns
        ----------
        myList : list in which all uncertainty elements produced from one element 
                 stores the resulting data

        """
        myList = []
        
        # uncertainty of about 3 percent applied
        numRange = (num + num*.0316227766) - (num - num*.0316227766)
        
        # generate increments to add to each number in uncertainty range
        divSize = numRange/(numDivisions - 1)
        
        # returing a list containing the element with uncertainty applied
        # going from a low to a high of about 3 percent below/above that element
        for x in range(0, numDivisions):
            myList.append((num - num*.0316227766) + x*divSize)
#             print(myList)
        return myList
    '''End function'''
    
    # determine which list to use based on the detector (l1 corresponds to detector 1)
    for i, sublist in enumerate(hitsArray):
        # det1
        if i == 0:
            useList = l1
        # det2
        elif i == 1:
            useList = l2
        # det3
        elif i == 2:
            useList = l3
        # det4
        else:
            useList = l4
            
        # for each hits occurence per hits list, apply uncertainty 
        # using the variable "steps" for the step size between low
        # and high values 
        
        steps = input('What do you want your step size to be between uncertainty low and high values?: ')
        for el in sublist:
            tempList = output_list(el, steps)
            for num in tempList:
                useList.append(num)

    resultArray = [l1,l2,l3,l4]
    resultArray = np.asarray(resultArray)
    
    # depending on the number of partitions used to generate 
    # uncertainty data ("steps"), for each row in extracted table, 
    # generate an array containing the angles to be stored with the
    # newly determined hits per each detector
    
    for i in range(0, len(trialArrayOH)):
        for x in range(0, steps):
            uncertAngle.append(trialArrayOH['Azimuth'][i])
    
    # to store data in strictly uncertainty table
    storeTemp = input('Enter 1 if you are storing data in the temp table and anything else if storing it in the regular table: ')
    
    if storeTemp == '1':
        t = Table.read('tempBurstData.txt', format='ascii.ecsv')
        for i in range(0, len(l1)):
            t.add_row([area['keV'][0], uncertAngle[i],
            area['ze'][0], resultArray[0][i],
            resultArray[1][i], resultArray[2][i],
            resultArray[3][i]])
            t.write('tempBurstData.txt', format='ascii.ecsv')
            
    # to store data in simulated GRB table (regular table)
    else:
        for i in range(0, len(l1)):
            t.add_row([area['keV'][0], uncertAngle[i],
            area['ze'][0], resultArray[0][i],
            resultArray[1][i], resultArray[2][i],
            resultArray[3][i]])
            t.write('storedBurstData.txt', format='ascii.ecsv')

else:
    pass
        

If the random burst location is to be determined, the energy of the burst is requested by the user, and all rows containing that select data are generated. The user is then asked the zenith of the random burst. The user is then asked which elements are to be extracted from the table using their indexes ("begin" and "end") to compare against the random burst. The least squares calculation is then applied to the entire set of selected data against the random burst: the position where the least squares results in the lowest value is placed in the variable, "result". "result" will then be used to store burst location, categorizing it by specfied energy ("energyRB") and zenith ("zenithRB").

In [7]:
# continue storing random burst data   
if randomB == '1':  
        
    inputDet = [[inputDet[0][0]], [inputDet[0][1]], [inputDet[0][2]], [inputDet[0][3]]]
    
    energyRB = input('What is the energy of the random burst?: ')
    if energyRB == '100':
        trialArray = t[t['Energy']==100]
    elif energyRB == '200':
        trialArray = t[t['Energy']==200]
    elif energyRB == '300':
        trialArray = t[t['Energy']==300]
    
    zenithRB = input('What is the desired zenith?: ')
    if zenithRB == '15':
        trialArray = trialArray[trialArray['Zenith']==15]
    elif zenithRB == '30':
        trialArray = trialArray[trialArray['Zenith']==30]
    elif zenithRB == '45':
        trialArray = trialArray[trialArray['Zenith']==45]
    elif zenithRB == '60':
        trialArray = trialArray[trialArray['Zenith']==60]
        
    begin = int(input('Enter the first index number in the table corresponding to the energy range: '))
    end = int(input('Enter the second index number in the table corresponding to the energy range: '))
    trialArray = trialArray[begin:end]
    
    # add a correction factor to make sure the table values line up
    begin = begin - 13
    end = end - 12
    
    print(trialArray)
    
    hitsArray = np.array([trialArray['hits1'], trialArray['hits2'], 
                                      trialArray['hits3'], trialArray['hits4']])
    print(hitsArray)
    print(np.array(inputDet))
        
    # use the least squares method to obtain the simulated GRB azimuth
    # that is most like the random GRB
    result = np.argmin(np.sum((hitsArray - inputDet)**2, axis=0))
    #     print(result) find position of angle in practiceTable
    
    resultDict = {'SourceDet1': inputDet[0], 'SourceDet2': inputDet[1], 
              'SourceDet3': inputDet[2], 'SourceDet4': inputDet[3],
              'energy': [trialArray['Energy'][0]], 
                  'zenith': [trialArray['Zenith'][0]],
              'azimuth': [trialArray['Azimuth'][result]], 'actAzimuth': area['az']}
else:
    pass
        

What is the energy of the random burst?: 100
What is the desired zenith?: 60
Enter the first number in the table corresponding to the energy range: 16
Enter the second number in the table corresponding to the energy range: 36
[[402 429 446 404 406 392 312 250 187 120 118 120 157 112 110 125 170 248
  346 392]
 [126 161 188 287 364 388 403 351 375 388 349 274 231 154 110 129 160 165
  129 143]
 [126 117 142 142 131 120 144 222 290 369 406 406 396 417 425 326 302 216
  180 131]
 [348 295 226 167 100 100 142 179 150 124 127 202 219 318 356 422 369 372
  347 335]]
[[254]
 [156]
 [227]
 [363]]


A series of options exists for the user in which case, if they are trying to determine the random burst location, they can either make a table for the first time to store the data, or they can add a new row, with their determined GRB info, to their pre-existing stored data.

In [9]:
begin = input('If you want to start determining the random burst direction for the first time enter 1, if you want to continue storing data enter 0, and 2 if you wish to pass: ')     
angleStore = input('Enter 1 to store angle match data normally and 2 to store it for least squares test: ')

# if making the result table for the first time, add determined GRB data
if begin == '1':
    
        if angleStore == '1':
            resultNames = ('SourceDet1', 'SourceDet2', 'SourceDet3', 'SourceDet4', 'energy',
                       'zenith', 'azimuth', 'actAzimuth')
            t2 = Table(resultDict, names=resultNames)
            t2.write('angleMatchTable.txt', format='ascii.ecsv')

        elif angleStore == '2':
            resultNames = ('SourceDet1', 'SourceDet2', 'SourceDet3', 'SourceDet4', 'energy',
                       'zenith', 'azimuth', 'actAzimuth')
            t2 = Table(resultDict, names=resultNames)
            t2.write('angleleastSQTestMatch.txt', format='ascii.ecsv')
    
# if updating result table, input all variables corresponding to determined GRB
elif begin == '0':
    
    if angleStore == '1':
        t2 = Table.read('angleMatchTable.txt', format='ascii.ecsv')
        # t2.write('resultTable.txt', format='ascii.ecsv')
        t2.add_row([resultDict['SourceDet1'], resultDict['SourceDet2'], resultDict['SourceDet3'],
                   resultDict['SourceDet4'], resultDict['energy'],
                   resultDict['zenith'], resultDict['azimuth'], resultDict['actAzimuth']])
        t2.write('angleMatchTable.txt', format='ascii.ecsv')

    elif angleStore == '2':

        t2 = Table.read('angleleastSQTestMatch.txt', format='ascii.ecsv')
        # t2.write('resultTable.txt', format='ascii.ecsv')
        t2.add_row([resultDict['SourceDet1'], resultDict['SourceDet2'], resultDict['SourceDet3'],
                   resultDict['SourceDet4'], resultDict['energy'],
                   resultDict['zenith'], resultDict['azimuth'], resultDict['actAzimuth']])
        t2.write('angleleastSQTestMatch.txt', format='ascii.ecsv')
        
# if you do not want to store any new found GRB, pass       
elif begin == '2':
    pass

If you want to start determining the random burst direction for the first time enter 1, if you want to continue storing data enter 0, and 2 if you wish to pass: 0
Enter 1 to store angle match data normally and 2 to store it for least squares test: 1
